In [13]:
import pandas as pd
import numpy as np
import os

# UKB

In [5]:
mgrm_file = "/data/jerrylee/pjt/BIGFAM.v.2.0/data/other-methods/GREML/bksk/UKB/mgrm-dor1.txt"
gcta_file = "/data/jerrylee/tools/gcta64"
pheno_path = "/data/jerrylee/pjt/BIGFAM.v.2.0/data/UKB/phenotype"
out_path = "/data/jerrylee/pjt/BIGFAM.v.2.0/data/other-methods/GREML/bksk/UKB"

In [6]:
fns = os.listdir(pheno_path)
len(fns)

106

In [7]:
for fn in fns:
    pheno = fn.split(".")[0]
    pheno_file = os.path.join(pheno_path, fn)
    out_file = f"{out_path}/{pheno}-bksk"
    cmd = f"{gcta_file} --reml --mgrm {mgrm_file} --pheno {pheno_file} --out {out_file} --thread-num 10"
    os.system(cmd)
    break

*******************************************************************
* Genome-wide Complex Trait Analysis (GCTA)
* version 1.94.0 beta Linux
* (C) 2010-present, Jian Yang, The University of Queensland
* Please report bugs to Jian Yang <jian.yang.qt@gmail.com>
*******************************************************************
Analysis started at 17:08:47 KST on Mon Dec 02 2024.
Hostname: h4

Accepted options:
--reml
--mgrm /data/jerrylee/pjt/BIGFAM.v.2.0/data/other-methods/GREML/bksk/UKB/mgrm-dor1.txt
--pheno /data/jerrylee/pjt/BIGFAM.v.2.0/data/UKB/phenotype/White_blood_cell__leukocyte__count.phen
--out /data/jerrylee/pjt/BIGFAM.v.2.0/data/other-methods/GREML/bksk/UKB/White_blood_cell__leukocyte__count-bksk
--thread-num 10

Note: the program will be running on 10 threads.

Reading phenotypes from [/data/jerrylee/pjt/BIGFAM.v.2.0/data/UKB/phenotype/White_blood_cell__leukocyte__count.phen].
Non-missing phenotypes of 444198 individuals are included from [/data/jerrylee/pjt/BIGFAM.v.2.0/da

# GS

In [13]:
mgrm_file = "/data/jerrylee/pjt/BIGFAM.v.2.0/data/other-methods/GREML/bksk/GS/mgrm.txt"
gcta_file = "/data/jerrylee/tools/gcta64"
pheno_path = "/data/jerrylee/pjt/BIGFAM.v.2.0/data/GS/phenotype"
out_path = "/data/jerrylee/pjt/BIGFAM.v.2.0/data/other-methods/GREML/bksk/GS"

In [14]:
fns = os.listdir(pheno_path)
len(fns)

40

In [15]:
for fn in fns:
    pheno = fn.split(".")[0]
    pheno_file = os.path.join(pheno_path, fn)
    out_file = f"{out_path}/{pheno}-bksk"
    cmd = f"{gcta_file} --reml --mgrm {mgrm_file} --pheno {pheno_file} --out {out_file} --thread-num 10"
    os.system(cmd)
    break

*******************************************************************
* Genome-wide Complex Trait Analysis (GCTA)
* version 1.94.0 beta Linux
* (C) 2010-present, Jian Yang, The University of Queensland
* Please report bugs to Jian Yang <jian.yang.qt@gmail.com>
*******************************************************************
Analysis started at 17:18:41 KST on Mon Dec 02 2024.
Hostname: h4

Accepted options:
--reml
--mgrm /data/jerrylee/pjt/BIGFAM.v.2.0/data/other-methods/GREML/bksk/GS/mgrm.txt
--pheno /data/jerrylee/pjt/BIGFAM.v.2.0/data/GS/phenotype/QT_interval.phen
--out /data/jerrylee/pjt/BIGFAM.v.2.0/data/other-methods/GREML/bksk/GS/QT_interval-bksk
--thread-num 10

Note: the program will be running on 10 threads.

Reading phenotypes from [/data/jerrylee/pjt/BIGFAM.v.2.0/data/GS/phenotype/QT_interval.phen].
Non-missing phenotypes of 19704 individuals are included from [/data/jerrylee/pjt/BIGFAM.v.2.0/data/GS/phenotype/QT_interval.phen].

There are 2 GRM file names specified in the

# merge

In [42]:
path = "/data/jerrylee/pjt/BIGFAM.v.2.0/data/other-methods/GREML/bksk"

In [43]:
def get_h2(df):
    h2_snp = df[df["Source"] == "V(G1)/Vp"]["Variance"].values[0]
    se_h2_snp = df[df["Source"] == "V(G1)/Vp"]["SE"].values[0]
    h2_ped_minus_h2_snp = df[df["Source"] == "V(G2)/Vp"]["Variance"].values[0]
    se_h2_ped_minus_h2_snp = df[df["Source"] == "V(G2)/Vp"]["SE"].values[0]
    
    h2_ped = h2_snp + h2_ped_minus_h2_snp
    se_h2_ped = np.sqrt(se_h2_snp**2 + se_h2_ped_minus_h2_snp**2)
    
    return h2_snp, se_h2_snp, h2_ped, se_h2_ped

In [44]:
df_gs = pd.DataFrame(columns=[
    "cohort", "pheno", "param", "median", "lower(2.5%)", "upper(97.5%)"])

path_cohort = f"{path}/GS"
fns = os.listdir(path_cohort)

fns = [fn for fn in fns if fn.endswith(".hsq")]
for fn in fns:
    pheno = fn.split(".")[0].split("-")[0]
    tmp = pd.read_csv(f"{path_cohort}/{fn}", sep='\t')
    
    h2_snp, se_h2_snp, h2_ped, se_h2_ped = get_h2(tmp)
    # h2_snp
    df_gs.loc[len(df_gs)] = ["GS", pheno, "h2_snp", h2_snp, h2_snp - 1.96 * se_h2_snp, h2_snp + 1.96 * se_h2_snp]
    
    # h2_ped
    df_gs.loc[len(df_gs)] = ["GS", pheno, "h2_ped", h2_ped, h2_ped - 1.96 * se_h2_ped, h2_ped + 1.96 * se_h2_ped]
        

In [45]:
df_gs

,cohort,pheno,param,median,lower(2.5%),upper(97.5%)
0,GS,expected,h2_snp,0.463129,0.423180,0.503078
1,GS,expected,h2_ped,0.793087,0.731625,0.854549
2,GS,avg_hr,h2_snp,0.158233,0.118727,0.197739
3,GS,avg_hr,h2_ped,0.311218,0.248506,0.373930
4,GS,PR_interval,h2_snp,0.000001,-0.034571,0.034573
...,...,...,...,...,...,...
71,GS,FEV,h2_ped,0.473966,0.400557,0.547375
72,GS,absi,h2_snp,0.021537,-0.303927,0.347001
73,GS,absi,h2_ped,0.527002,0.013161,1.040843
74,GS,FVC,h2_snp,0.299135,0.252432,0.345838


In [52]:
df_ukb = pd.DataFrame(columns=[
    "cohort", "pheno", "param", "median", "lower(2.5%)", "upper(97.5%)", "n_meta"])

path_cohort = f"{path}/UKB"
fns = os.listdir(path_cohort)

fns = [fn for fn in fns if fn.endswith(".hsq")]
phenos = set([fn.split(".")[0].split("-")[0] for fn in fns])

def ivw(means, ses):
    w = 1 / np.square(ses)
    w_sum = np.sum(w)
    mean = np.sum(means * w) / w_sum
    se = np.sqrt(1 / w_sum)
    return mean, se

for pheno in phenos:
    pheno_fns = []
    for dor in [1, 2, 3]:
        fn = f"{pheno}-{dor}-bksk.hsq"
        if fn in fns:
            pheno_fns.append(fn)
        else:
            print(f"{fn} not found")
    
    # if len(pheno_fns) != 3:
    #     continue
    
    snp_means = []
    snp_ses = []
    ped_means = []
    ped_ses = []
    for fn in pheno_fns:
        fn_with_path = f"{path_cohort}/{fn}"
        tmp = pd.read_csv(fn_with_path, sep='\t')
        h2_snp, se_h2_snp, h2_ped, se_h2_ped = get_h2(tmp)
        snp_means.append(h2_snp)
        snp_ses.append(se_h2_snp)
        ped_means.append(h2_ped)
        ped_ses.append(se_h2_ped)
        
    snp_mean, snp_se = ivw(snp_means, snp_ses)
    ped_mean, ped_se = ivw(ped_means, ped_ses)
    
    df_ukb.loc[len(df_ukb)] = [
        "UKB", pheno, "h2_snp", snp_mean, snp_mean - 1.96 * snp_se, snp_mean + 1.96 * snp_se, len(pheno_fns)]

    df_ukb.loc[len(df_ukb)] = [
        "UKB", pheno, "h2_ped", ped_mean, ped_mean - 1.96 * ped_se, ped_mean + 1.96 * ped_se, len(pheno_fns)]
        

Immature_reticulocyte_fraction-2-bksk.hsq not found
Immature_reticulocyte_fraction-3-bksk.hsq not found
Alanine_aminotransferase-2-bksk.hsq not found
Alanine_aminotransferase-3-bksk.hsq not found
Vitamin_D-2-bksk.hsq not found
Vitamin_D-3-bksk.hsq not found
Apolipoprotein_A-3-bksk.hsq not found
Mean_reticulocyte_volume-2-bksk.hsq not found
Mean_reticulocyte_volume-3-bksk.hsq not found
Lymphocyte_percentage-2-bksk.hsq not found
Lymphocyte_percentage-3-bksk.hsq not found
Cystatin_C-2-bksk.hsq not found
Cystatin_C-3-bksk.hsq not found
LDL_direct-2-bksk.hsq not found
LDL_direct-3-bksk.hsq not found
Trunk_fat_percentage-2-bksk.hsq not found
Trunk_fat_percentage-3-bksk.hsq not found
C-1-bksk.hsq not found
C-2-bksk.hsq not found
C-3-bksk.hsq not found
Sodium_in_urine-2-bksk.hsq not found
Sodium_in_urine-3-bksk.hsq not found
Monocyte_count-2-bksk.hsq not found
Monocyte_count-3-bksk.hsq not found
Impedance_of_leg__left_-2-bksk.hsq not found
Impedance_of_leg__left_-3-bksk.hsq not found
Eosinophi

/tmp/ipykernel_445482/329847085.py:13: RuntimeWarning: invalid value encountered in scalar divide
  mean = np.sum(means * w) / w_sum
/tmp/ipykernel_445482/329847085.py:14: RuntimeWarning: divide by zero encountered in scalar divide
  se = np.sqrt(1 / w_sum)


Impedance_of_leg__right_-2-bksk.hsq not found
Impedance_of_leg__right_-3-bksk.hsq not found
Seated_height-2-bksk.hsq not found
Seated_height-3-bksk.hsq not found
Red_blood_cell__erythrocyte__count-2-bksk.hsq not found
Red_blood_cell__erythrocyte__count-3-bksk.hsq not found
Weight-2-bksk.hsq not found
Weight-3-bksk.hsq not found
Mean_corpuscular_haemoglobin_concentration-2-bksk.hsq not found
Mean_corpuscular_haemoglobin_concentration-3-bksk.hsq not found
Apolipoprotein_B-2-bksk.hsq not found
Apolipoprotein_B-3-bksk.hsq not found
Glycated_haemoglobin__HbA1c_-3-bksk.hsq not found
Mean_platelet__thrombocyte__volume-2-bksk.hsq not found
Mean_platelet__thrombocyte__volume-3-bksk.hsq not found
Urate-2-bksk.hsq not found
Urate-3-bksk.hsq not found
Arm_fat_mass__left_-2-bksk.hsq not found
Arm_fat_mass__left_-3-bksk.hsq not found
Arm_fat_mass__right_-2-bksk.hsq not found
Arm_fat_mass__right_-3-bksk.hsq not found
Neutrophill_percentage-2-bksk.hsq not found
Neutrophill_percentage-3-bksk.hsq not fo

In [53]:
df = pd.concat([df_gs, df_ukb.drop(columns=["n_meta"])]).reset_index(drop=True)
df

,cohort,pheno,param,median,lower(2.5%),upper(97.5%)
0,GS,expected,h2_snp,0.463129,0.423180,0.503078
1,GS,expected,h2_ped,0.793087,0.731625,0.854549
2,GS,avg_hr,h2_snp,0.158233,0.118727,0.197739
3,GS,avg_hr,h2_ped,0.311218,0.248506,0.373930
4,GS,PR_interval,h2_snp,0.000001,-0.034571,0.034573
...,...,...,...,...,...,...
275,UKB,Reticulocyte_percentage,h2_ped,0.120735,0.045651,0.195819
276,UKB,Alkaline_phosphatase,h2_snp,0.239357,0.187619,0.291095
277,UKB,Alkaline_phosphatase,h2_ped,0.504769,0.421546,0.587992
278,UKB,Leg_fat,h2_snp,NaN,NaN,NaN


In [54]:
df.to_csv(
    f"{path}/GREML-bksk.tsv",
    sep='\t',
    index=False
)